In [1]:
import pandas as pd
import numpy as np

In [5]:
guests = pd.read_excel("guests.xlsx", index_col=0)
guests.drop("guest", axis= 1, inplace = True)
guests = 1 - guests 
vec_discount = guests["discount"].values #converts the discount column to a vector

FileNotFoundError: [Errno 2] No such file or directory: 'guests.xlsx'

In [3]:
hotels = pd.read_excel(hotels.xlsx", index_col=0)
vec_prices = hotels["price"].values

SyntaxError: EOL while scanning string literal (<ipython-input-3-5cd8fa6c26b3>, line 1)

In [4]:
preferences = pd.read_excel("C:/Users/oanam/Desktop/hotels/preferences.xlsx", index_col=0)
preferences["guest"] = pd.to_numeric(preferences["guest"].str.replace('guest_','')) - 1
preferences["hotel"] = pd.to_numeric(preferences["hotel"].str.replace('hotel_','')) - 1
pref_pivoted = pd.pivot_table(preferences, index ='guest', columns ='hotel', values= "priority" , aggfunc="min") #it pivots the original dataframe to get to a matrix-like format
rank_matrix = pref_pivoted.rank(axis = 1, method="dense").values #reorders preferences

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/oanam/Desktop/hotels/preferences.xlsx'

In [ ]:
#This function creates a utility matrix from the ranked preferences matrix
def to_utility_matrix(rank_matrix):
  user_utility = []
  for row in rank_matrix:
    utility_values = np.linspace(start=1, stop= 0.1, num=len(set(row[~np.isnan(row)]))) #utility goes from 1 to 0.1 split evenly between preferences
    user_utility.append([utility_values[int(i)-1] if i == i else i for i in row ]) #i == i to check if i is not nan 
  return np.nan_to_num(np.array(user_utility), nan = 0)

In [ ]:
utility_matrix = to_utility_matrix(rank_matrix)

In [ ]:
# def print_output(res, guest_count, room_count, hotel_count, t):
#   res_avg = np.round_(np.array(res).mean(axis=0), 3)
#   print(f"With {t} iterations, on average we have: \n{res_avg[0]}/{guest_count} guests placed \n{res_avg[1]}/{room_count} rooms occupied \n{res_avg[2]}/{hotel_count} hotels occupied \n{res_avg[3]} total revenue \n{res_avg[4]} total satisfaction")

In [ ]:
def results(choice_matrix, vec_prices=vec_prices, vec_discount=vec_discount, utility_matrix=utility_matrix):
  guests_placed = choice_matrix.sum(axis=1).sum()
  rooms_occupied = choice_matrix.sum(axis=0).sum()
  hotels_occupied = np.where(choice_matrix.sum(axis=0) > 0, 1, 0).sum()
  prices_matrix = vec_prices * choice_matrix #all 1s in the matrix will be replaced by that hotel's price
  discount_matrix = np.multiply(prices_matrix, vec_discount[:, np.newaxis]) #the room price is then discounted for each user
  revenue = discount_matrix.sum()
  utility = np.multiply(choice_matrix, utility_matrix).sum()
  return([guests_placed, rooms_occupied, hotels_occupied, revenue, utility])

In [ ]:
def random_model(hotels, vec_prices, vec_discount, rank_matrix, t=1000):
  guest_count, room_count, hotel_count = rank_matrix.shape[0] , hotels["rooms"].sum(), rank_matrix.shape[1]
  least = min([guest_count, room_count])
  res = []
  vec_rooms = np.repeat(hotels["price"], hotels["rooms"]).index.values #Denormalized the dataframe by adding rows. This vec contains the hotel number as many times as its rooms 
  for _ in range(t):
    choice_matrix = np.zeros(rank_matrix.shape) #creates filled with zeros 4000 x 400
    random_rooms = np.random.choice(vec_rooms, least, replace=False ) #selects randomly 4000 rooms from all possible rooms
    choice_matrix[tuple(range(least)),tuple(random_rooms)] = 1 #assign 1 to all occupied rooms given their indices
    res.append(results(choice_matrix, vec_prices, vec_discount))
  return list(np.array(res).mean(axis=0))

In [ ]:
def preference_model(hotels, vec_prices, rank_matrix):
  rank_matrix_filled, choice_matrix  = np.nan_to_num(rank_matrix), np.zeros(rank_matrix.shape) 
  rooms = hotels["rooms"].values
  for idx, row in enumerate(rank_matrix_filled):
    c = np.multiply(row, np.where(rooms > 0, 1, 0) ) #sets preferences values to 0 where there is no room availability
    try:
      user_choice = np.where(c == np.min(c[np.nonzero(c)]), 1, 0) #The minimum nonzero is converted to 1 and the rest to 0
      choice_matrix[(idx),] = user_choice
      rooms = rooms - user_choice #subtract 1 from the room availability where it's now occupied 
    except ValueError: #pass if array are all zeros
     pass
  return  results(choice_matrix)

In [ ]:
random_model(hotels, vec_prices, vec_discount, rank_matrix, t=1000)

In [ ]:
preference_model(hotels, vec_prices, rank_matrix)

In [ ]:
hotels_by_price = hotels.sort_values('price') #order by price the hotels
hotels_by_price["hotel"] = pd.to_numeric(hotels_by_price["hotel"].str.replace('hotel_','')) - 1
hotel_price = hotels_by_price["hotel"].values #vector hotel in order by price
rooms_price = hotels_by_price["rooms"].values #vector rooms in order by price
price_asc = hotels_by_price["price"].values #vector prices in ascending order
guest_arranged = []
vec_price_guest = []
hotel_occ = 0

for i in list(range(len(hotel_price))):
    hotel_to_consider = hotel_price[i]
    rooms_free = rooms_price[i]
    vec_pref = pref_pivoted[hotel_price[i]] #vector of preferences by hotel
    price_room = price_asc[i] #price of room_i
    hotel_occ += 1 #count hotel
    
    while rooms_free > 0: #if there are free rooms
        for j in list(range(int(vec_pref.min()),int(vec_pref.max()))):
            guest_with_pref = vec_pref[vec_pref == j].index #name of guests with preferences
            for pref in list(range(len(guest_with_pref))):
                for arr in list(range(len(guest_arranged))):
                    if guest_with_pref[pref] != guest_arranged[arr]:
                        guest_arranged = guest_arranged.append(guest_with_pref[pref])
                        vec_price_guest = vec_price_guest.append(price_asc[i])
                        rooms_free = rooms_free - 1
                        
        guest_no_pref = vec_pref.index[vec_pref.apply(np.isnan)] #name of guests with no prefernce
        for pref in list(range(len(guest_no_pref))):
                for arr in list(range(len(guest_arranged))):
                    if guest_no_pref[pref] != guest_arranged[arr]:
                        guest_arranged = guest_arranged.append(guest_no_pref[pref])
                        vec_price_guest = vec_price_guest.append(price_asc[i])
                        rooms_free = rooms_free - 1
    
for i in list(range(len(guest_arranged))):
    vec_disc_byprice = guests[guests.index == guest_arranged[i]].values #discount by price
        
revenue_by_price = vec_price_guest * vec_disc_byprice #revenue
print(guest_arranged, hotel_occ)
#len(guest_arranged) number of clients arranged = number of rooms occupied
#hotel_occ number of different hotels occupied
#degree of customer satisfaction...

In [ ]:
hotels_by_price = hotels.sort_values('price') #order by price the hotels
hotels_by_price["hotel"] = pd.to_numeric(hotels_by_price["hotel"].str.replace('hotel_','')) - 1
hotel_price = hotels_by_price["hotel"].values #vector hotel in order by price
rooms_price = hotels_by_price["rooms"].values #vector rooms in order by price
price_asc = hotels_by_price["price"].values #vector prices in ascending order
guest_arranged = []
vec_price_guest = []
hotel_occ = 0